In [1]:

from pinecone import Pinecone, PodSpec
from dotenv import dotenv_values

env = dotenv_values()


c:\Users\aloys\Documents\Github\Hackomania\venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:

pc = Pinecone(api_key=env['pc_api_key'])

pc.create_index(
    name="test-scheme",
    dimension=128,
    metric="cosine",
    spec=PodSpec(
        environment="gcp-starter",
    ),
)



ForbiddenException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': 'd23b5e8981cae49f237a5bd11256a69e', 'Date': 'Sun, 31 Mar 2024 08:30:03 GMT', 'Server': 'Google Frontend', 'Content-Length': '101', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"FORBIDDEN","message":"Projects only support a single Starter index."},"status":403}


In [6]:

from openai import OpenAI
from dotenv import load_dotenv, dotenv_values
load_dotenv()

client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", "")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

text = "you"
print(len(get_embedding(text)))


1536


In [1]:

from typing import List, Dict

from langchain_text_splitters import MarkdownHeaderTextSplitter, MarkdownTextSplitter
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.vectorstores import VectorStore

from pinecone import Pinecone

from openai import OpenAI
from dotenv import load_dotenv, dotenv_values
load_dotenv()
env = dotenv_values()

embeddings: OpenAIEmbeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vectorstore_chunk: VectorStore = PineconeVectorStore(index_name=env["PINECONE_INDEX_NAME_CHUNK"], embedding=embeddings)
# vectorstore_whole: VectorStore = PineconeVectorStore(index_name=env["PINECONE_INDEX_NAME_WHOLE"], embedding=embeddings)

mdsplitter: MarkdownHeaderTextSplitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ],
    # strip_headers=False,
)

with open("data/Energy Efficient Grant.md", "r") as f:
    data: str = f.read()

split_text: List[Document] = mdsplitter.split_text(data)
headers: Dict[str, str] = {"Header 3": "###", "Header 2": "##", "Header 1": "#"}
header_split_text: List[Document] = []
for text_item in split_text:
    text: str = text_item.page_content
    for header, v in headers.items():
        if header not in text_item.metadata.keys():
            continue
        text = f"{v} {text_item.metadata[header]}\n" + text

    header_split_text.append(
        Document(
            page_content=text,
            meta_data={
                "title": "Energy Efficient Grant",
                "source": "https://www.gobusiness.gov.sg/energy-efficiency-grant/?src=about-govassist",
            },
        )
    )

vectorstore_chunk.add_documents(header_split_text)
# vectorstore_whole.add_documents(
#     [
#         Document(
#             page_content=data,
#             metadata={
#                 "title": "Energy Efficient Grant",
#                 "source": "https://www.gobusiness.gov.sg/energy-efficiency-grant/?src=about-govassist",
#             },
#         ),
#     ]
# )


c:\Users\aloys\Documents\Github\Hackomania\venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


['be451cab-3a73-4829-b6a8-2b4b29ff55bf',
 '29f2fcfb-bb55-4a4c-b035-b8ee3fe11802',
 'd291c53d-1155-44eb-8616-3a88b2113d7b',
 '22a4b514-e4ea-4268-b33b-44261ca51b0e']

In [8]:

chunk_retriever = vectorstore_chunk.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.4, "k": 3}
    # search_type="similarity", search_kwargs={"k": 2}
    # search_type="mmr"
)

docs = chunk_retriever.get_relevant_documents("Who can apply for Energy Efficient Grant?")

for i, doc in enumerate(docs):
    print(i+1)
    print(doc.page_content)
    print()


1
# The Energy Efficient Grant
The Energy Efficient Grant (EEG) was launched in September 2022 and aims to help businesses within the Food and Service, Food Manufacturing or Retail sectors to cope with rising energy costs, through co-funding investments in more energy efficient equipment.  
It provides financial support for SMEs to adopt pre-approval energy efficient equipment in the following categories: LED lighting, air-conditioning, cooking hobs, refrigerators, water heaters and clothes dryers.

2
# The Energy Efficient Grant
## EEG Eligibility
### How to apply and submit EEG claims?
4 steps:
1. Access the list of pre-approved energy efficient equipment (brands & models) and identify relevant eqiupment that best suits your business needs.
2. Source for the energy efficient equipment and get a quotation from the vendor.
3. Submit an application on the Business Grants Portal (BGP). You will need to register for a Corppass account to transact on the portal
4. After your application ha